# Redis' publisher Implementation

In event-driven architectures, efficient message broadcasting is essential for real-time communication between different components of a system. Redis' `publisher` implementation enables seamless event propagation using its Pub/Sub and Streams features. This approach is particularly useful in scenarios where multiple consumers need to act upon published messages in a distributed environment. The advantages of using publisher are Low Latency Communication, Scalability, Decoupled Architecture, Event-Driven Processing, and Efficient Data Streaming, making it ideal for event logging. For more information about Redis's sub/pub please check this [reference](https://redis.io/docs/latest/develop/interact/pubsub/).

**Objectives:**

In this notebook, the user will:
- Set up a Redis connection to ensure reliable communication.
- Define a function to publish events using Redis Streams.
- Simulate event publishing with sample data.
- Integrate with a real-time event source, such as STAC API, to dynamically generate messages.
- Test message delivery to verify that the publisher correctly sends data to Redis streams.

### Table of content:
- [Import dependencies](#import-dependencies)
- [Pub's connection and publish](#pubs-connection-and-publish)
- [STAC API Query for Event Discovery](#stac-api-query-for-event-discovery)

## Import dependencies

In [1]:
from os import environ
from redis import Redis
from time import sleep
import pystac_client
from datetime import datetime, timedelta
from time import sleep
from loguru import logger

## Pub's connection and publish

The following script the user define some function such as:
- `connect_to_redis`: This function set up a redis connection to ensure reliable communication
- `send_event`: This function responsible for publishing events using Redis stream

In [2]:
stream_key = environ.get("STREAM", "STREAM")
producer = environ.get("PRODUCER", "project-a")


def connect_to_redis():
    hostname = environ.get("REDIS_HOSTNAME", "redis-service") 
    port = environ.get("REDIS_PORT", 6379)

    return Redis(hostname, port, retry_on_timeout=True)


def send_event(redis_connection, reference):
    count = 0

    try:
        # TODO cloud events
        # un-map the "data" wrt app package parameters
        data = {
            "subject": reference,
            "producer": producer,
            "href": reference,
        }
        resp = redis_connection.xadd(stream_key, data)
        print(resp)
        count += 1

    except ConnectionError as e:
        logger.error(f"ERROR REDIS CONNECTION: {e}")



In the cell below, the user create a connection object to Redis.

In [3]:

connection = connect_to_redis()


The code below iterates over a desired list of Sentinel-2 satellite imagery references which are hosted on **Earth Search AWS Element84**. It then sends each reference as an event to a Redis stream using the `send_event` function. Additionally, the sleep(1) function will take effect, introducing a 1-second delay between each event to prevent overwhelming the Redis server. This setup enables **event-driven satellite imagery processing**, facilitating real-time geospatial analysis.

In [ ]:
references = [
    "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_10TFK_20211230_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2D_10TFK_20210708_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20210708_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20210718_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2D_10TFK_20220524_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220514_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220504_0_L2A"
]

for reference in references:
    send_event(connection, reference)
    #break
    sleep(1)


## STAC API Query for Event Discovery

The script below queries the Earth Search AWS STAC API to retrieve Sentinel-2 satellite imagery for a specified geographic region and time range. It dynamically generates **10 event dates** at **5-day intervals**, further dividing each date into **four 6-hour time windows** to refine temporal resolution. Using these time constraints, it performs batch queries to fetch available imagery, extracts relevant metadata, and publishes the results as events to Redis for downstream processing. The implementation ensures efficient API usage prevents request overload with controlled pauses (`sleep()`), and enables event-driven geospatial data processing for applications such as environmental monitoring, disaster response, and etc.

In [ ]:
# Define the STAC API URL
stac_api_url = "https://earth-search.aws.element84.com/v1"

# Create a client instance
client = pystac_client.Client.open(stac_api_url)

# Array of event dates (ISO 8601 format)
start_date = datetime.fromisoformat("2021-07-04T00:00:00Z".replace("Z", "+00:00"))

# Number of dates and interval in days
num_dates = 10
interval_days = 5

event_dates = [date.strftime("%Y-%m-%dT%H:%M:%SZ") for date in [start_date + timedelta(days=i * interval_days) for i in range(num_dates)]]
logger.info(f'len event_dates: {len(event_dates)}')
# Loop over each event date
for idx, event_date_str in enumerate(event_dates):
    # Parse the event date
    logger.info(f"Event Date {idx+1}: {event_date_str}")
    event_date = datetime.fromisoformat(event_date_str.replace("Z", "+00:00"))
    
    # Query four times with 6-hour steps
    for i in range(4):
        # Calculate the start and end times for this step
        start_time = event_date - timedelta(minutes=(6 * 60 * (i + 1)))
        end_time = event_date - timedelta(minutes=(6 * 60 * i) + 1)
        
        
        # Format the time range in ISO 8601 format
        time_of_interest = [start_time.strftime('%Y-%m-%dT%H:%M:%SZ') , end_time.strftime('%Y-%m-%dT%H:%M:%SZ')]
        
        print(time_of_interest)

        # Perform the search query
        search = client.search(
            collections=["sentinel-2-l2a"],
            bbox="-121.399,39.834,-120.74,40.472",
            datetime=time_of_interest,
            max_items=50  # Limit the number of returned items
        )
        
        # Fetch the search results
        items = list(search.items())
        
        # Display some basic information about the found items
        print(f"Results for event date: {event_date_str}, Interval {i+1}")
        print(f"Time of interest: {time_of_interest}")
        if items:
            for item in items:
                print(f"ID: {item.id}")
                print(f"Date: {item.datetime}")
                print(f"Assets: {list(item.assets.keys())}")
                print(f"Bounding box: {item.bbox}")
                print()
                send_event(connection, [link.href for link in item.links if link.rel in ["self"]][0])
        else:
            logger.warning("No results found.")
        print("-" * 40)
        sleep(5)
    print("-" * 120)

    sleep(90)
    